In [1]:
# import of packages
from proj03 import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.stats.weightstats import ttest_ind

# autoreload for easier debugging
%load_ext autoreload
%autoreload 2

In [6]:
# import data
baseline = pd.read_stata('data/baseline.dta')
BOK_inflation = pd.read_stata('data/BOK_inflation.dta')
cleanPriceData_Y1Y2 = pd.read_stata('data/cleanPriceData_Y1Y2.dta')
intensity_obs_short = pd.read_stata('data/intensity_obs_short.dta')
LRFU_select_dataset = pd.read_stata('data/LRFU_select_dataset.dta')
MS1MS2_pooled = pd.read_stata('data/MS1MS2_pooled.dta')
repayment_dataY1 = pd.read_stata('data/repayment_dataY1.dta')

In [11]:
print(list(baseline.columns))

['oafid', 's2_4_homemadeof', 's2_5_home_roof', 's2_6_dwelling_floor', 'num_rooms', 'num_children', 's3_2_13_av_sell_01', 's3_2_13_av_sell_02', 's3_2_13_av_sell_03', 's3_2_13_av_sell_04', 's3_2_13_av_sell_05', 's3_2_13_av_sell_06', 's3_2_13_av_sell_07', 's3_2_13_av_sell_08', 's3_2_13_av_sell_09', 's3_2_13_av_sell_10', 's3_2_13_av_sell_11', 's3_2_13_av_sell_12', 's3_2_13_av_purch01', 's3_2_13_av_purch02', 's3_2_13_av_purch03', 's3_2_13_av_purch04', 's3_2_13_av_purch05', 's3_2_13_av_purch06', 's3_2_13_av_purch07', 's3_2_13_av_purch08', 's3_2_13_av_purch09', 's3_2_13_av_purch10', 's3_2_13_av_purch11', 's3_2_13_av_purch12', 's3_2_13_av_01', 's3_2_13_av_02', 's3_2_13_av_03', 's3_2_13_av_04', 's3_2_13_av_05', 's3_2_13_av_06', 's3_2_13_av_07', 's3_2_13_av_08', 's3_2_13_av_09', 's3_2_13_av_10', 's3_2_13_av_11', 's3_2_13_av_12', 'digit_recall', 'site', 'treatment', 'tags', 'lockbox', 'loantakeup_roster', 'baselined', 'gender', 'num_adults', 'finished_primary', 'finished_secondary', 'num_schoolch

In [3]:
# Load dataset
ms1ms2_pooled = pd.read_stata('data/MS1MS2_pooled.dta')

# Filter rows where MS != 2
ms1ms2_pooled = ms1ms2_pooled[ms1ms2_pooled['MS'] == 2]

# Keep specific columns
ms1ms2_pooled = ms1ms2_pooled[['oafid', 'treatMS1MS2']]

# Count rows
print(ms1ms2_pooled.shape[0])

# Collapse/aggregation
ms1ms2_pooled = ms1ms2_pooled.groupby('oafid', as_index=False).sum()

# Count rows after collapse
print(ms1ms2_pooled.shape[0])

# Rename column
ms1ms2_pooled.rename(columns={'treatMS1MS2': 'treat13'}, inplace=True)



2993
1019


In [5]:
# Load another dataset
baseline = pd.read_stata('data/baseline.dta')

# Modify 'delta' column
baseline['delta'] = 1 - baseline['delta']

# Keep specific columns
columns_to_keep = ['oafid', 'logtotcons_base', 'male', 'num_adults', 'num_schoolchildren', 'finished_primary',
                   'finished_secondary', 'cropland', 'num_rooms', 'schoolfees', 'totcons_base', 'logpercapcons_base',
                   'total_cash_savings_base', 'total_cash_savings_trimmed', 'has_savings_acct', 'taken_bank_loan',
                   'taken_informal_loan', 'liquidWealth', 'wagepay', 'businessprofitmonth', 'price_avg_diff_pct',
                   'price_expect_diff_pct', 'harvest2011', 'netrevenue2011', 'netseller2011', 'autarkic2011',
                   'maizelostpct2011', 'harvest2012', 'correct_interest', 'digit_recall', 'maizegiver', 'delta', 'treatment']
baseline = baseline[columns_to_keep]

# Rename columns
baseline.columns = [col + '_base' if col != 'oafid' and col != 'treatment' else col for col in baseline.columns]
baseline.rename(columns={'oafid_base': 'oafid', 'treatment_base': 'treatment2012'}, inplace=True)

# Generate new column based on condition
baseline['treat12'] = baseline['treatment2012'].apply(lambda x: x in ['T1', 'T2'])

# Replace treat12 with NaN where treatment2012 is empty
baseline['treat12'] = baseline['treat12'].where(baseline['treatment2012'] != '', other=pd.NA)

# Merge with the previous dataset
merged_data = pd.merge(baseline, ms1ms2_pooled, on='oafid', how='left', indicator=True)

# Further processing based on merge results
merged_data['in_sample_Y2'] = merged_data['_merge'] == 'both'
merged_data['newin13'] = merged_data['_merge'] == 'right_only'
merged_data['attrit13'] = merged_data['_merge'] == 'left_only'

# Drop rows based on merge results
merged_data = merged_data[merged_data['_merge'] != 'right_only']


KeyError: 'delta'

In [ ]:
import pandas as pd
from statsmodels.stats.weightstats import ttest_ind

# Assuming 'merged_data' is your DataFrame after all previous steps

# Define the variables list for which you want to perform t-tests
vars = ['male_base', 'num_adults_base', 'num_schoolchildren_base', 'finished_primary_base', 
        'finished_secondary_base', 'cropland_base', 'num_rooms_base', 'schoolfees_base', 
        'totcons_base', 'logpercapcons_base', 'total_cash_savings_base', 'total_cash_savings_trimmed_base',
        'has_savings_acct_base', 'taken_bank_loan_base', 'taken_informal_loan_base', 'liquidWealth_base', 
        'wagepay_base', 'businessprofitmonth_base', 'price_avg_diff_pct_base', 'price_expect_diff_pct_base',
        'harvest2011_base', 'netrevenue2011_base', 'netseller2011_base', 'autarkic2011_base', 
        'maizelostpct2011_base', 'harvest2012_base', 'correct_interest_base', 'digit_recall_base', 'maizegiver_base', 'delta_base']

# Create an empty DataFrame to store test results
test_results = pd.DataFrame(columns=['Variable', 'Mean_Group1', 'Mean_Group2', 'Observations', 'T-Statistic', 'P-Value'])

for var in vars:
    group1 = merged_data[merged_data['in_sample_Y2']][var].dropna()
    group2 = merged_data[~merged_data['in_sample_Y2']][var].dropna()
    
    if not group1.empty and not group2.empty:
        t_stat, p_value, df = ttest_ind(group1, group2, usevar='unequal')
        test_results = test_results.append({'Variable': var, 'Mean_Group1': group1.mean(), 'Mean_Group2': group2.mean(),
                                            'Observations': len(group1) + len(group2), 'T-Statistic': t_stat, 'P-Value': p_value},
                                           ignore_index=True)

# Convert the DataFrame to LaTeX
latex_table = test_results.to_latex(index=False, float_format="%.2f")

print(latex_table)
